In [ ]:

from sqlmodel import Session, create_engine, text, SQLModel, select, update, delete
import os
from dotenv import load_dotenv

load_dotenv()

# Paths
MEDIA_PATH = os.path.join('../', os.getenv('MEDIA_PATH'))
DB_PATH = os.path.join('../', os.getenv('DB_PATH'))
engine = create_engine(f"sqlite:///{DB_PATH}")

print(MEDIA_PATH)
print(DB_PATH)



In [ ]:

def update_table():
    with Session(engine) as session:
        session.exec(text("ALTER TABLE media_items ADD COLUMN user_deleted BOOLEAN DEFAULT false"))
        session.commit()


In [ ]:

def fix_extension_types():
    with Session(engine) as session:
        media_items = session.exec(select(MediaItem)).all()
        updated_count = 0
        for item in media_items:
            extension = os.path.splitext(item.file_name)[1]
            if extension in ['mp4', 'webm'] and item.type != 'video':
                item.type = 'video'
                updated_count += 1
                print(f"Updated entry {item.id}: {item.file_name} from {item.type} to video")

            file_path = os.path.join(MEDIA_PATH, item.file_name)
            if os.path.exists(file_path):
                _, file_extension = os.path.splitext(file_path)
                if file_extension.lower() != '.mp4' and item.type == 'video':
                    new_file_name = f"{os.path.splitext(item.file_name)[0]}.mp4"
                    new_file_path = os.path.join(MEDIA_PATH, new_file_name)
                    os.rename(file_path, new_file_path)
                    item.file_name = new_file_name
                    item.url = f'/media/{new_file_name}'
                    updated_count += 1
                    print(f"Renamed file and updated entry {item.id}: {item.file_name} to {new_file_name}")

        session.commit()
        print(f"Updated {updated_count} entries")


In [ ]:

def set_viewed_true():
    with Session(engine) as session:
        session.exec(update(MediaItem).values(seen=True))
        session.commit()


In [ ]:
def mark_files_deleted():
    with Session(engine) as session:
        query = select(MediaItem).where(MediaItem.user_deleted == False)
        updated_count = 0
        for item in session.exec(query):
            target_file = os.path.join(MEDIA_PATH, item.file_name)
            if not os.path.exists(target_file):
                item.user_deleted = True
                updated_count += 1
                print(f"Missing entry {item.id} - file {item.file_name}")
        session.commit()
        print(f"Updated {updated_count} entries")


In [ ]:
def mark_favorites_from_downloads():
    with Session(engine) as session:
        static_files = {item.name: item for item in os.scandir('../static/media') if item.is_file()}
        for item in os.scandir('/mnt/e/Downloads'):
            if item.is_file() and item.name in static_files:
                media_item = session.exec(select(MediaItem).where(MediaItem.file_name == item.name)).first()
                if media_item:
                    print("Found!")
                    media_item.favorite = True
        session.commit()